In [ ]:
import itk

# ----------------------
# Lecture de l'image
# ----------------------
input_filename = './images/brain.png'
image = itk.imread(input_filename)
ImageType = type(image) # On récupère le type de l'image
# ----------------------
# Création et paramétrage du filtre
# ----------------------
smoothFilter = itk.SmoothingRecursiveGaussianImageFilter[ImageType,ImageType].New() 
# On créé une instance du filtre qui prend en entrée et ressort des images du même type que l'image d'entrée
smoothFilter.SetInput(image) # On spécifie l'image d'entrée du filtre
sigma = 1 #Ecart-type
smoothFilter.SetSigma(sigma) # On sélectionne la variance du filtre
# ----------------------
# Écriture de la sortie du filtre
# ----------------------
output_filename = './images_result/myImage_smoothed.png'
itk.imwrite(smoothFilter.GetOutput(), output_filename) # On écrit  sur le disque la sortie du filtre Gaussien
image_smoothed = itk.imread(output_filename)
difference_filter = itk.AbsoluteValueDifferenceImageFilter[ImageType,type(image_smoothed),ImageType].New()
difference_filter.SetInput1(image)
difference_filter.SetInput2(image_smoothed)
difference_filter.Update()
image_difference = difference_filter.GetOutput()
itk.imwrite(image_difference, './images_result/myImage_difference.png') # On écrit ssur le disque la sortie de la différence des 2 images




In [37]:
import itk

# ----------------------
# Lecture des images
# ----------------------
input_fixed_filename = 'images\BrainProtonDensitySliceBorder20.png'
input_moving_filename = 'images\BrainProtonDensitySliceShifted13x17y.png'
fixedImage = itk.imread(input_fixed_filename)
movingImage = itk.imread(input_moving_filename)
fixed_image_type = type(fixedImage) # On récupère le type de l'image fixe
moving_image_type = type(movingImage) #On récupère le type de l'image mouvante

#-------------------------
# Recalage par translation
# ------------------------

# Paramétrage

#Création de l'optimiseur
optimizer = itk.RegularStepGradientDescentOptimizer.New() # Instance de la classe d'optimiseur choisie
# optimizer.SetMaximumStepLength( 10 ) # Borne supérieure du pas de descente (en pixel)
# optimizer.SetMinimumStepLength( 1 ) # Borne inférieure du pas de descente (en pixel)
optimizer.SetNumberOfIterations( 100 ) # Nombre maximal d'itération
#Création de la transformée
initialTransform = itk.TranslationTransform[itk.D, 2].New() # Instance de la classe de transformation choisie
initialParameters = initialTransform.GetParameters() # Récupération des paramètres de la transformation
initialParameters[0] = 0.0 # Translation selon x
initialParameters[1] = 0.0 # Translation selon y
# Création de l'interpolateur
interpolator = itk.LinearInterpolateImageFunction[moving_image_type,itk.D].New()  # Instance de l'interpolateur
# Création de la métrique
metric = itk.MeanSquaresImageToImageMetric[fixed_image_type,moving_image_type].New()    # Instance du métrique

# Exécution du recalage
registration_filter = itk.ImageRegistrationMethod[fixed_image_type,moving_image_type].New() # Instance de la classe de recalage
registration_filter.SetFixedImage( fixedImage ) # Image de référence
registration_filter.SetMovingImage( movingImage ) # Image à recaler
registration_filter.SetOptimizer( optimizer ) # Optimiseur
registration_filter.SetTransform( initialTransform ) # Transformation
registration_filter.SetInitialTransformParameters(initialParameters) # Application de la transformation initiale
registration_filter.SetInterpolator( interpolator ) # Interpolateur
registration_filter.SetMetric( metric ) # Métrique
registration_filter.Update() # Exécution du recalage

# Récupération et écriture des résultats
final_transform = registration_filter.GetTransform()
resample_filter = itk.ResampleImageFilter[fixed_image_type,moving_image_type].New() # Instance de la classe de ré-échantillonnage
resample_filter.SetInput( fixedImage ) # Image d'entrée
resample_filter.SetTransform(final_transform)
resample_filter.SetSize(fixedImage.GetLargestPossibleRegion().GetSize())
recaledImage =resample_filter.GetOutput()
itk.imwrite(recaledImage, 'image_recalee.png')


# # Différence entre image d'entrée et image recalée
# diff_filter = itk.AbsoluteValueDifferenceImageFilter[fixed_image_type,moving_image_type,fixed_image_type].New()
# diff_filter.SetInput1(fixedImage)
# diff_filter.SetInput2(recaledImage)
# diff_filter.Update()
# image_difference = diff_filter.GetOuput()
# itk.imwrite(image_difference, './images_result/recaledImage_difference.png') # On écrit sur le disque la sortie de la différence des 2 images

# Analyse de la métrique à convergence
valueMetric = optimizer.GetValue()
# Nombre d'itérations
nbIterations = optimizer.GetNumberOfIterations()
# Valeurs des paramètres de transformation
param1 = initialTransform.GetParameters()
print('Paramètres de la transaltion (x,y) :')
print(param1.GetElement(0),param1.GetElement(1))

# #Image de différence entre image recalée et image de référence
rescale = itk.RescaleIntensityImageFilter[type(recaledImage),fixed_image_type].New()
rescale.SetInput(recaledImage)
rescale.Update()
image_diff = rescale.GetOutput()
itk.imwrite(image_diff,'./images_result/recaledImage_diff.png')



Paramètres de la transaltion (x,y) :
2.373289096642497 -4.266387268572226
